### All paths used in this notebook are Relative, thus the folder structure should not be altered



In [ ]:
#required libraries , uncomment to install

! pip install pandas --q
! pip install numpy  --q
! pip install spacy  --q

In [ ]:
#mount the drive for uploading the folder with model and input file.
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
model_path = '/content/drive/MyDrive/model-best-updated'

In [ ]:
#import the required libraries
import pandas as pd 
import numpy as np 
import spacy #framework used for building the ML model
import os 
from spacy import displacy
import json
import random
import argparse
import pickle
from spacy.tokens import DocBin
import warnings
from spacy import displacy
warnings.filterwarnings("ignore")
pd.set_option("display.max_columns",None)
pd.set_option("display.max_rows",None)
pd.set_option("display.max_colwidth",2)

In [ ]:
# model is loaded from the folder model-best-updated

model=spacy.load(model_path) 

In [ ]:
#The input file example should be like below:
"""

{
    "tweet_id":"TWEET ID",
    "text":"YOUR TWEET TEXT GOES HERE"
}

"""

# specify the path for the input file (JSONL)

upload_path="/content/train.jsonl" #path for the input file


In [ ]:
def open_json(path):
  mode='r'
  dict_list = []
  with open(path, mode) as data:
    for jobject in data:
      jdict = json.loads(jobject)
      dict_list.append(jdict)

  return dict_list

In [ ]:
#function to compute the start and end offset of the location mention indentified
def find_index(sentence,word):
    str2=word
    str1=sentence
    start_index=str1.index(str2)
    length_word=len(str2)
    end_index=start_index+length_word
    start=start_index
    end=end_index
    return start,end

In [ ]:
#store the data tweet and tweet id in the variable
data_c = open_json(upload_path)

data = []
tweet_id = []

for d in data_c:
  data.append(d['text'])
  tweet_id.append(d['tweet_id'])

In [ ]:
# compute the output results
# data_located_list=""
twitter_id=tweet_id

all_locations = []

for data_item in data:
  test_preds=model(data_item) #find recognition
  list_locations = []
  for entity in test_preds.ents:
    if entity.label_=='Location':
      list_locations.append(entity.text)
  all_locations.append(list_locations)

In [ ]:
DICT_LIST = []
for a, locs in zip(data, all_locations):
  dict_list = []
  for b in locs:
    data_located_Dict={}
    data_located_Dict['text']=b
    start_index, end_index = find_index(a, b) 
    data_located_Dict['start_offset'] = start_index
    data_located_Dict['end_offset'] = end_index
    dict_list.append(data_located_Dict)
  DICT_LIST.append(dict_list)
  
  
                    
#Final Results
Model_Results = []
for tid, dicts in zip(tweet_id, DICT_LIST):
  Model_Results.append({"tweet_id":tid,"location_mentions":dicts})

In [ ]:
results = []

for item in Model_Results:
  results.append(item.__str__() +'\n')

In [ ]:
# print the results

with open("output.jsonl", "a") as f:
   f.writelines(results)
   f.close()

In [ ]:
#The other codes for Data loading,data preparation,cleaning and modelling are available in the docker images
# use this command :   docker pull 19991806/ingenuity_lmr_geoai